In [69]:
import numpy as np
import pandas as pd
from scipy.ndimage.interpolation import shift

In [32]:
class timetable:
    # Arrays for calculating the costs
    cost_multiplier = np.array([0,0,9,9,9,18,18,36,36,235])
    cost_additive = np.array([0,50,50,100,200,200,300,300,400,500])

    cost_multiplier_matrix = np.resize(cost_multiplier, (100,10))
    cost_additive_matrix = np.resize(cost_additive, (100, 10))

    # Init a 100 day by 5000 families by 10 choices matrix as the timetable
    def __init__(self):
        self.timetable = np.zeros((100, 5000, 10))
        
    # Function for assigning a family their specific choice
    def assign_family_by_choice(self, fam_id, choice):
        self.timetable[:,fam_id,:] = np.zeros((100,10))
        day = df.iloc[fam_id, choice + 1]
        self.timetable[day + 1, fam_id, choice] = df.loc[fam_id, 'n_people']
    
    # Function for assigning a family a specific day
    def assign_family_by_day(self, fam_id, day):
        self.timetable[:,fam_id,:] = np.zeros((100,10))
        choice = np.where(df.iloc[fam_id] == day)[0][0]
        self.timetable[day + 1, fam_id, choice] = df.loc[fam_id, 'n_people']
        
    # Function for calculating the preference cost to santa of the current timetable
    def get_preference_cost(self):
        multiple_cost = np.sum(self.timetable, axis=1) * self.cost_multiplier_matrix
        additive_cost = np.count_nonzero(self.timetable, axis=1) * self.cost_additive_matrix
        
        return np.sum(multiple_cost + additive_cost)
    
    # Function for calculating the tax cost to Santa
    def get_tax_cost(self):
        occupancy_days = np.sum(np.sum(self.timetable, axis=1), axis=1)
        prior_day_occupancy = shift(occupancy_days, -1, cval=100)
        
        return np.sum(((occupancy_days-125)/400)*(occupancy_days**(0.5 + (abs(occupancy_days - prior_day_occupancy) / 50))))
    
    # Function for calculating the total cost to Santa
    def get_total_cost(self):
        return self.get_tax_cost() + self.get_preference_cost()

In [33]:
df = pd.read_csv('family_data.csv')
df.head()

,family_id,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people
0,0,52,38,12,82,33,75,64,76,10,28,4
1,1,26,4,82,5,11,47,38,6,66,61,4
2,2,100,54,25,12,27,82,10,89,80,33,3
3,3,2,95,1,96,32,6,40,31,9,59,2
4,4,53,1,47,93,26,3,46,16,42,39,4


In [34]:
# Create timetable object
schedule = timetable()

In [38]:
# assign a family one of their choices
schedule.assign_family_by_choice(fam_id=0, choice=3)
print(schedule.timetable[83,0])

[0. 0. 0. 4. 0. 0. 0. 0. 0. 0.]


In [39]:
# calculate the preference cost
schedule.get_preference_cost()

136.0

In [30]:
schedule.timetable[:,0,:] = np.zeros((100,10))

In [78]:
occupancy_days = np.sum(np.sum(schedule.timetable, axis=1), axis=1)
prior_day_occupancy = shift(occupancy_days, -1, cval=100)

In [80]:
np.sum(((occupancy_days-125)/400)*(occupancy_days**(0.5 + (abs(occupancy_days - prior_day_occupancy) / 50))))

-0.6759587185336929

In [65]:
occupancy_days[-1:]+occupancy_days[:-1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 4., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [74]:
occupancy_days

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 4., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [68]:
[occupancy_days[-1]] + occupancy_days[:-1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 4., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [76]:
shift(occupancy_days, -1, cval=100)

array([ 1.01262585e-61, -3.77787337e-61,  1.40988676e-60, -5.26175971e-60,
        1.71479460e-59, -7.32868486e-59,  2.53596594e-58, -1.02075412e-57,
        3.80950624e-57, -1.29427973e-56,  5.30595771e-56, -1.98021038e-55,
        6.57458268e-55, -2.75807726e-54,  1.02932845e-53, -3.58049388e-53,
        1.43366958e-52, -5.35052771e-52,  1.99684412e-51, -6.78413255e-51,
        2.78124508e-50, -1.03797479e-49,  3.87377467e-49, -1.44571239e-48,
        4.71124432e-48, -2.01361760e-47,  7.51492317e-47, -2.80460751e-46,
        9.59112618e-46, -3.55598972e-45,  1.31772651e-44, -5.44079400e-44,
        1.80632421e-43, -7.57804845e-43,  2.82816619e-42, -1.05548599e-41,
        3.93912734e-41, -1.47010234e-40,  5.02731913e-40, -2.23125940e-39,
        6.90702002e-39, -2.85192275e-38,  1.18190150e-37, -3.97221597e-37,
        1.48245118e-36, -5.53258313e-36,  1.91432486e-35, -7.10404112e-35,
        2.87587888e-34, -1.07329261e-33,  4.00558255e-33, -1.49490376e-32,
        5.57905677e-32, -